This workbook parses the set of excel files that contain the daily route informations that was given by the city of port coquitlam.

In [ ]:
import pandas as pd
import datetime
import re
import glob

In [ ]:
path = r'C:\Users\Aidan\OneDrive - Simon Fraser University (1sfu)\Garbage Route Optimization\Weekly Weight and Tips' # use your path
all_files = glob.glob(path + "/*.xls")
li = []
for filename in all_files:
    df = pd.read_excel(filename,
                  header=[0,1])
    li.append(df)

In [ ]:
for i in range(0,len(li)):
    li[i].columns=li[i].columns.to_flat_index()
    li[i] = li[i].iloc[1:,]

In [ ]:
# Creating data frame

# This contains granular info
dailyInfo = pd.DataFrame(columns=['fileName',
                                  'date',
                                  'zone',
                                  'driver',
                                  'truckNumber',
                                  'garbageTips',
                                  'garbageWeight',
                                  'garbageTime',
                                 'recyclingTips',
                                 'recyclingWeight',
                                 'recyclingTime',
                                 'greenTips',
                                 'greenWeight',
                                 'greenTime'])

# Note I dont bother with the totals since they can be calculated from the above dataframe

In [ ]:
# Regulard expression to capture the zone
zoneExpression = re.compile(r'[Zz]one\s*\S*')
customerExpression = re.compile(r'[Cc]ustomer')


for i in range(0,len(li)):
    # Getting the df we are working with
    df = li[i]
    
    # Getting the filename for reference
    fileName = all_files[0][all_files[0].rfind('\\')+1:]
    
    # Zeroing these out, these need to be non to detect when we need to look ahead to find the zone
    date = None
    zone = None

    for i, row in df.iterrows():    
        # Extracting Values
        rowValues = list(row)
        
        # Testing for the last row, if yes we stop
        if all(pd.isnull(i) for i in rowValues):
            break

        # These only get zeroed out when the date/zone changes
        # This is also detecting rows that contain totals, we aren't recording these as they can be easily calculated 
        # using the data set
        if customerExpression.match(str(rowValues[0])):
            date = None
            zone = None
            continue

        # Extracting the current date
        if isinstance(rowValues[0], datetime.datetime):
            date = rowValues[0]     


        if zone == None:
            zoneValues = [ word for i, word in enumerate(list(df.iloc[i,:])) if zoneExpression.match(str(word))]
            if len(zoneValues) > 0:
                zone = zoneValues[0]


        driver = rowValues[1]
        truckNumber = rowValues[2]
        garbageTips = rowValues[3]
        garbageWeight = rowValues[4]
        garbageTime = rowValues[5]
        recyclingTips = rowValues[6]
        recyclingWeight = rowValues[7]
        recyclingTime = rowValues[8]
        greenTips = rowValues[9]
        greenWeight = rowValues[10]
        greenTime = rowValues[11]

        # Just skipping some useless rows here (its the greyed rows stuff in the excel files)
        if zone == None:
            continue

        dailyInfo = dailyInfo.append({'fileName':fileName,
                                      'date':date,
                                      'zone':zone,
                                      'driver':driver,
                                      'truckNumber':truckNumber,
                                      'garbageTips':garbageTips,
                                      'garbageWeight':garbageWeight,
                                      'garbageTime':garbageTime,
                                      'recyclingTips':recyclingTips,
                                      'recyclingWeight':recyclingWeight,
                                      'recyclingTime':recyclingTime,
                                      'greenTips':greenTips,
                                      'greenWeight':greenWeight,
                                      'greenTime':greenTime},ignore_index=True)


        # Zeroing out values to prevent bad data
        if customerExpression.match(str(rowValues[0])):
            date = None
            zone = None

In [ ]:
dailyInfo.to_csv('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/dailyinfo.csv')

In [12]:
df = pd.read_csv('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/dailyinfo.csv',index_col=0)

In [20]:
collectionTime = df.groupby(['date','zone']).sum()[['garbageTime','greenTime','recyclingTime']]

In [21]:
numTrucks = df.dropna(subset=['driver']).groupby(['date','zone']).count()['driver']

In [22]:
timeByTrucks = collectionTime.join(numTrucks)
timeByTrucks = timeByTrucks.rename(columns = {'driver':'numTrucks'})
garbage = timeByTrucks[['garbageTime','numTrucks']].rename(columns={'garbageTime':'time'})
green = timeByTrucks[['greenTime','numTrucks']].rename(columns={'greenTime':'time'})
re = timeByTrucks[['recyclingTime','numTrucks']].rename(columns={'recyclingTime':'time'})
d = pd.concat([garbage,green,re])
d = d.dropna()
d = d[d['time'] > 0]
d.groupby(['zone','numTrucks']).mean().reset_index().pivot(index='numTrucks',columns='zone',values='time')

zone,Zone 1,Zone 2,Zone 3,Zone 4,Zone 5
numTrucks,,,,,
4,NaN,18.000000,29.000000,NaN,17.379310
5,24.090909,24.555556,20.000000,27.50,18.368421
6,24.151515,24.166667,23.520548,23.85,21.944444
7,24.562500,25.200000,22.772727,23.00,23.750000


In [23]:
d.groupby(['zone','numTrucks']).median().reset_index().pivot(index='numTrucks',columns='zone',values='time')

zone,Zone 1,Zone 2,Zone 3,Zone 4,Zone 5
numTrucks,,,,,
4,NaN,18.0,28.0,NaN,16.0
5,20.0,20.0,20.0,25.0,18.0
6,24.0,24.0,24.0,24.0,24.0
7,24.0,24.0,24.0,24.0,24.0


In [24]:
d.groupby(['zone','numTrucks']).max().reset_index().pivot(index='numTrucks',columns='zone',values='time')

zone,Zone 1,Zone 2,Zone 3,Zone 4,Zone 5
numTrucks,,,,,
4,NaN,20.0,32.0,NaN,32.0
5,35.0,35.0,20.0,40.0,28.0
6,40.0,48.0,24.0,42.0,26.0
7,29.0,28.0,28.5,24.0,25.0


In [26]:
garbage = timeByTrucks[timeByTrucks['garbageTime'] > 0][['garbageTime','numTrucks']]
recycling = timeByTrucks[timeByTrucks['recyclingTime'] > 0][['recyclingTime','numTrucks']]
green = timeByTrucks[timeByTrucks['greenTime'] > 0][['greenTime','numTrucks']]

In [27]:
garbage.groupby(['numTrucks','zone']).mean()

garbageTime
numTrucks zone               
4         Zone 2    20.000000
          Zone 5    16.000000
5         Zone 1    20.000000
          Zone 2    18.666667
          Zone 3    20.000000
          Zone 4    20.000000
          Zone 5    17.250000
6         Zone 1    23.764706
          Zone 2    23.888889
          Zone 3    22.421053
          Zone 4    23.826087
          Zone 5    16.000000
7         Zone 1    24.000000
          Zone 2    26.666667
          Zone 3    24.000000
          Zone 4    24.000000
          Zone 5    24.000000

In [28]:
recycling.groupby(['numTrucks','zone']).mean()

recyclingTime
numTrucks zone                 
4         Zone 3      29.000000
          Zone 5      29.333333
5         Zone 1      35.000000
          Zone 2      35.000000
          Zone 4      35.000000
          Zone 5      28.000000
6         Zone 1      25.294118
          Zone 2      26.000000
          Zone 3      23.947368
          Zone 4      24.000000
          Zone 5      23.571429
7         Zone 1      25.250000
          Zone 2      24.000000
          Zone 3      18.166667
          Zone 4      22.666667
          Zone 5      24.000000

In [29]:
green.groupby(['numTrucks','zone']).mean()

greenTime
numTrucks zone             
4         Zone 2  16.000000
          Zone 5  16.000000
5         Zone 1  20.000000
          Zone 2  20.000000
          Zone 3  20.000000
          Zone 4  20.000000
          Zone 5  17.222222
6         Zone 1  23.750000
          Zone 2  23.388889
          Zone 3  23.885714
          Zone 4  23.789474
          Zone 5  22.000000
7         Zone 1  24.500000
          Zone 2  25.000000
          Zone 3  24.800000
          Zone 4  23.000000
          Zone 5  23.400000

In [ ]:
green.to_csv('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/greenTimes.csv')
garbage.to_csv('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/garbageTimes.csv')
recycling.to_csv('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/recyclingTimes.csv')